In [38]:
# ==============================================================================
# Bloco 1: Importação das bibliotecas e carregamento dos dados
# ==============================================================================
import pandas as pd
import numpy as np


# Para exibir os gráficos no próprio notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliotecas importadas com sucesso!")

# Carregando o arquivo principal com os resultados dos jogos
# O arquivo está na pasta 'data' que fica um nível acima da pasta 'notebooks'
try:
    df_full = pd.read_csv('../data/raw/campeonato-brasileiro-full.csv')
    print("Arquivo 'campeonato-brasileiro-full.csv' carregado com sucesso!")
    print(f"O dataset contém {df_full.shape[0]} partidas e {df_full.shape[1]} colunas.")
except FileNotFoundError:
    print("ERRO: O arquivo 'campeonato-brasileiro-full.csv' não foi encontrado.")
    print("Por favor, certifique-se de que ele está na pasta 'data' no diretório raiz do projeto.")

# Exibindo as primeiras linhas para entender a estrutura
if 'df_full' in locals():
    df_full.head()

Bibliotecas importadas com sucesso!
Arquivo 'campeonato-brasileiro-full.csv' carregado com sucesso!
O dataset contém 8785 partidas e 16 colunas.


In [39]:
# ==============================================================================
# Carregamento dos arquivos adicionais da pasta data
# ==============================================================================
# Carregando os outros arquivos disponíveis para análises mais detalhadas

print("Carregando arquivos adicionais...")

try:
    # Arquivo com dados de cartões
    df_cartoes = pd.read_csv('../data/raw/campeonato-brasileiro-cartoes.csv')
    print(f"✓ Cartões: {df_cartoes.shape[0]} registros, {df_cartoes.shape[1]} colunas")
except FileNotFoundError:
    print("✗ Arquivo de cartões não encontrado")
    df_cartoes = None

try:
    # Arquivo com estatísticas completas
    df_estatisticas = pd.read_csv('../data/raw/campeonato-brasileiro-estatisticas-full.csv')
    print(f"✓ Estatísticas: {df_estatisticas.shape[0]} registros, {df_estatisticas.shape[1]} colunas")
except FileNotFoundError:
    print("✗ Arquivo de estatísticas não encontrado")
    df_estatisticas = None

try:
    # Arquivo com dados de gols
    df_gols = pd.read_csv('../data/raw/campeonato-brasileiro-gols.csv')
    print(f"✓ Gols: {df_gols.shape[0]} registros, {df_gols.shape[1]} colunas")
except FileNotFoundError:
    print("✗ Arquivo de gols não encontrado")
    df_gols = None

print("\nTodos os arquivos de dados foram processados!")

Carregando arquivos adicionais...
✓ Cartões: 20953 registros, 8 colunas
✓ Estatísticas: 17570 registros, 13 colunas
✓ Gols: 9861 registros, 6 colunas

Todos os arquivos de dados foram processados!


In [40]:
# ==============================================================================
# Bloco 2: Limpeza e Preparação dos Dados
# ==============================================================================
# Vamos trabalhar com as colunas de placar. Elas podem não ser numéricas.
# Convertemos para número, e qualquer erro (ex: placar não preenchido) vira 'NaN' (Not a Number)
if 'df_full' in locals():
	df_full['mandante_Placar'] = pd.to_numeric(df_full['mandante_Placar'], errors='coerce')
	df_full['visitante_Placar'] = pd.to_numeric(df_full['visitante_Placar'], errors='coerce')

	# Removemos as linhas onde o placar não é um número (jogos que talvez não aconteceram ou dados faltantes)
	df_jogos_realizados = df_full.dropna(subset=['mandante_Placar', 'visitante_Placar']).copy()

	# Convertemos os placares para inteiros, pois agora sabemos que são todos números
	df_jogos_realizados['mandante_Placar'] = df_jogos_realizados['mandante_Placar'].astype(int)
	df_jogos_realizados['visitante_Placar'] = df_jogos_realizados['visitante_Placar'].astype(int)

	print("Limpeza dos dados de placar concluída.")
	print(f"Analisando um total de {len(df_jogos_realizados)} partidas com resultados válidos.")
else:
	print("ERRO: O dataframe 'df_full' não está definido. Certifique-se de que o arquivo foi carregado corretamente.")

Limpeza dos dados de placar concluída.
Analisando um total de 8785 partidas com resultados válidos.


In [41]:
# ==============================================================================
# Bloco 3: Limpeza e Refinamento Adicional do DataFrame (v3)
# ==============================================================================
if 'df_jogos_realizados' in locals():
    print("\n--- INICIANDO LIMPEZA ADICIONAL ---")
    
    # --- 1. REMOÇÃO DE COLUNAS DESNECESSÁRIAS OU VAZIAS ---
    percentual_nulos = df_jogos_realizados.isnull().sum() / len(df_jogos_realizados) * 100
    colunas_para_remover = percentual_nulos[percentual_nulos > 80].index.tolist()
    
    # Adicionamos colunas que são redundantes ou não serão usadas
    colunas_para_remover.extend(['ID', 'mandante_Estado', 'visitante_Estado', 'formacao_mandante', 'formacao_visitante', 'tecnico_mandante', 'tecnico_visitante'])
    
    # Garantir que as colunas existem antes de tentar remover
    colunas_existentes_para_remover = [col for col in colunas_para_remover if col in df_jogos_realizados.columns]
    
    df_jogos_realizados = df_jogos_realizados.drop(columns=colunas_existentes_para_remover)
    print(f"\nColunas removidas: {colunas_existentes_para_remover}")

    # --- 2. RENOMEAÇÃO DE COLUNAS ---
    df_jogos_realizados = df_jogos_realizados.rename(columns={'rodata': 'rodada'})
    # A linha que convertia tudo para minúsculo foi REMOVIDA
    print("\nColuna 'rodata' corrigida para 'rodada'.")

    # --- 3. TRATAMENTO DE DATAS E HORAS (MANTENDO SEPARADAS) ---
    # Converte a coluna 'data' para o formato datetime
    df_jogos_realizados['data'] = pd.to_datetime(df_jogos_realizados['data'], errors='coerce', dayfirst=True)
    
    # Remove linhas onde a data não pôde ser convertida
    df_jogos_realizados = df_jogos_realizados.dropna(subset=['data'])
    print("\nColuna 'data' convertida para o formato datetime.")
    
    # Limpa a coluna 'hora', preenche valores nulos e garante que seja texto (string)
    df_jogos_realizados['hora'] = df_jogos_realizados['hora'].fillna('00:00').astype(str)
    print("Coluna 'hora' limpa e padronizada como texto.")
    
    print("\n--- LIMPEZA ADICIONAL CONCLUÍDA ---")
    display(df_jogos_realizados.head())
    df_jogos_realizados.info()
    
else:
    print("ERRO: O dataframe 'df_jogos_realizados' não está definido.")


--- INICIANDO LIMPEZA ADICIONAL ---

Colunas removidas: ['ID', 'mandante_Estado', 'visitante_Estado', 'formacao_mandante', 'formacao_visitante', 'tecnico_mandante', 'tecnico_visitante']

Coluna 'rodata' corrigida para 'rodada'.

Coluna 'data' convertida para o formato datetime.
Coluna 'hora' limpa e padronizada como texto.

--- LIMPEZA ADICIONAL CONCLUÍDA ---


,rodada,data,hora,mandante,visitante,vencedor,arena,mandante_Placar,visitante_Placar
0,1,2003-03-29,16:00,Guarani,Vasco,Guarani,Brinco de Ouro,4,2
1,1,2003-03-29,16:00,Athletico-PR,Gremio,Athletico-PR,Arena da Baixada,2,0
2,1,2003-03-30,16:00,Flamengo,Coritiba,-,Maracanã,1,1
3,1,2003-03-30,16:00,Goias,Paysandu,-,Serra Dourada,2,2
4,1,2003-03-30,16:00,Internacional,Ponte Preta,-,Beira Rio,1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rodada            8785 non-null   int64         
 1   data              8785 non-null   datetime64[ns]
 2   hora              8785 non-null   object        
 3   mandante          8785 non-null   object        
 4   visitante         8785 non-null   object        
 5   vencedor          8785 non-null   object        
 6   arena             8785 non-null   object        
 7   mandante_Placar   8785 non-null   int64         
 8   visitante_Placar  8785 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 617.8+ KB


In [42]:
# ==============================================================================
# Bloco 3.1: Verificação e Ajuste da Coluna 'vencedor'
# ==============================================================================
if 'df_jogos_realizados' in locals():
    print("--- Iniciando verificação de consistência da coluna 'vencedor' ---")

    # Condições baseadas nos placares
    condicao_mandante_venceu = df_jogos_realizados['mandante_Placar'] > df_jogos_realizados['visitante_Placar']
    condicao_visitante_venceu = df_jogos_realizados['visitante_Placar'] > df_jogos_realizados['mandante_Placar']
    condicao_empate = df_jogos_realizados['mandante_Placar'] == df_jogos_realizados['visitante_Placar']

    # Verificando inconsistências
    # 1. Jogos onde o mandante venceu no placar, mas não consta como vencedor na coluna 'vencedor'
    inconsistencia1 = df_jogos_realizados[condicao_mandante_venceu & (df_jogos_realizados['vencedor'] != df_jogos_realizados['mandante'])]
    
    # 2. Jogos onde o visitante venceu no placar, mas não consta como vencedor
    inconsistencia2 = df_jogos_realizados[condicao_visitante_venceu & (df_jogos_realizados['vencedor'] != df_jogos_realizados['visitante'])]

    # 3. Jogos que empataram no placar, mas o vencedor não é '-'
    inconsistencia3 = df_jogos_realizados[condicao_empate & (df_jogos_realizados['vencedor'] != '-')]

    total_inconsistencias = len(inconsistencia1) + len(inconsistencia2) + len(inconsistencia3)

    if total_inconsistencias == 0:
        print("✓ Verificação concluída: A coluna 'vencedor' é consistente com os placares.")
    else:
        print(f"✗ ATENÇÃO: Foram encontradas {total_inconsistencias} inconsistências entre os placares e a coluna 'vencedor'.")

    # --- Ajuste para Melhor Legibilidade ---
    # Substituindo o hífen '-' pela palavra 'Empate' para clareza
    df_jogos_realizados.loc[df_jogos_realizados['vencedor'] == '-', 'vencedor'] = 'Empate'
    
    print("\nAjuste realizado: O valor '-' foi substituído por 'Empate' na coluna 'vencedor'.")

    # Exibindo a contagem de resultados para confirmar a mudança
    print("\nContagem de resultados após o ajuste:")
    print(df_jogos_realizados['vencedor'].value_counts().head())

else:
    print("ERRO: O dataframe 'df_jogos_realizados' não está definido.")

--- Iniciando verificação de consistência da coluna 'vencedor' ---
✓ Verificação concluída: A coluna 'vencedor' é consistente com os placares.

Ajuste realizado: O valor '-' foi substituído por 'Empate' na coluna 'vencedor'.

Contagem de resultados após o ajuste:
vencedor
Empate           2322
Sao Paulo         382
Flamengo          374
Internacional     357
Palmeiras         351
Name: count, dtype: int64


In [43]:
# ==============================================================================
# Bloco 4: Filtragem do Período (2012-2024)
# ==============================================================================
if 'df_jogos_realizados' in locals():

    ano_inicio = 2012
    ano_fim = 2024

    print(f"Total de jogos antes do filtro: {len(df_jogos_realizados)}")

    # Filtra o DataFrame para manter apenas os jogos dentro do intervalo de anos desejado
    df_jogos_realizados = df_jogos_realizados[
        (df_jogos_realizados['data'].dt.year >= ano_inicio) &
        (df_jogos_realizados['data'].dt.year <= ano_fim)
    ].copy() # .copy() é importante para evitar avisos futuros

    print(f"Total de jogos após o filtro ({ano_inicio}-{ano_fim}): {len(df_jogos_realizados)}")

    # Exibindo o primeiro e o último jogo do novo dataset para confirmar o filtro
    print("\nPrimeiro jogo do período:")
    display(df_jogos_realizados.head(1))

    print("\nÚltimo jogo do período:")
    display(df_jogos_realizados.tail(1))

else:
    print("ERRO: O dataframe 'df_jogos_realizados' não está definido.")

Total de jogos antes do filtro: 8785
Total de jogos após o filtro (2012-2024): 4939

Primeiro jogo do período:


,rodada,data,hora,mandante,visitante,vencedor,arena,mandante_Placar,visitante_Placar
3846,1,2012-05-19,18:30,Palmeiras,Portuguesa,Empate,Pacaembu,1,1



Último jogo do período:


,rodada,data,hora,mandante,visitante,vencedor,arena,mandante_Placar,visitante_Placar
8784,38,2024-12-08,16:05,Gremio,Corinthians,Corinthians,Arena do Grêmio,0,3


In [44]:
# ==============================================================================
# Bloco 5: Salvando o DataFrame Processado
# ==============================================================================
if 'df_jogos_realizados' in locals():
    # Define o caminho para a pasta de dados processados
    caminho_processed = '../data/processed'
    
    # Cria a pasta se ela não existir
    if not os.path.exists(caminho_processed):
        os.makedirs(caminho_processed)
        print(f"\nDiretório '{caminho_processed}' criado.")

    # Define o caminho completo do arquivo a ser salvo
    caminho_salvar = os.path.join(caminho_processed, 'df_jogos_realizados.pkl')
    
    # Salva o dataframe em formato pickle
    df_jogos_realizados.to_pickle(caminho_salvar)

    print(f"\nDataFrame limpo e preparado salvo com sucesso em: '{caminho_salvar}'")
else:
    print("\nERRO: DataFrame 'df_jogos_realizados' não existe, arquivo não foi salvo.")



DataFrame limpo e preparado salvo com sucesso em: '../data/processed\df_jogos_realizados.pkl'
